In [1]:
# !pip install nsetools

In [2]:
# pip install ta

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import re
import random
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from yahooquery import Ticker
import datetime as dt
from dateutil.relativedelta import relativedelta
from ta.volatility import BollingerBands, AverageTrueRange
from ta.trend import EMAIndicator
from ta.volume import MFIIndicator
import yfinance as yf
import logging

# Data Preperation

In [50]:
nifty_50 = pd.read_csv(r'C:\Users\Preetham S Gowda\Desktop\Algo Trading\Market Research\Factor Investing\factor model\ind_nifty50list.csv')

In [51]:
symbols=nifty_50['Symbol']+'.NS'

In [53]:
def fin_bal_data_create(tickers):
    financial_data = pd.DataFrame()

    for ticker in tickers:
        try:
            stock = Ticker(ticker)
            temp_df = stock.all_financial_data('q')

            if temp_df is None or len(temp_df) == 0:
                print(f"No data for {ticker}")
                continue

            temp_df = pd.DataFrame(temp_df)
            temp_df.reset_index(inplace=True)

            financial_data = pd.concat([financial_data, temp_df], axis=0)

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return financial_data

In [54]:
financial_data = fin_bal_data_create(symbols)

Error fetching data for DUMMYITC.NS: DataFrame constructor not properly called!


In [55]:
# Convert to datetime if not already
financial_data['asOfDate'] = pd.to_datetime(financial_data['asOfDate'])

# Get min and max dates
min_date = financial_data['asOfDate'].min()
max_date = financial_data['asOfDate'].max()

# Calculate the difference in months
diff = relativedelta(max_date, min_date)
total_quarters = (diff.years * 12 + diff.months)//3

print(min_date)
print(max_date)
print(total_quarters)

2023-09-30 00:00:00
2025-03-31 00:00:00
6


In [56]:
for col in financial_data.columns:
    print(f"{col}: {financial_data[col].isna().sum()/len(financial_data)}")

symbol: 0.0
asOfDate: 0.0
periodType: 0.0
currencyCode: 0.0
AccountsPayable: 0.5344262295081967
AccountsReceivable: 0.5672131147540984
AccumulatedDepreciation: 0.819672131147541
AdditionalPaidInCapital: 0.8065573770491803
AllowanceForDoubtfulAccountsReceivable: 0.8754098360655738
AssetsHeldForSaleCurrent: 0.8131147540983606
AvailableForSaleSecurities: 0.5180327868852459
BasicAverageShares: 0.18360655737704917
BasicEPS: 0.18360655737704917
BuildingsAndImprovements: 0.8360655737704918
CapitalLeaseObligations: 0.5967213114754099
CapitalStock: 0.46229508196721314
CashAndCashEquivalents: 0.4459016393442623
CashCashEquivalentsAndShortTermInvestments: 0.5508196721311476
CashEquivalents: 0.8590163934426229
CashFinancial: 0.7377049180327869
CommonStock: 0.46229508196721314
CommonStockEquity: 0.4459016393442623
ConstructionInProgress: 0.5868852459016394
CostOfRevenue: 0.3770491803278688
CurrentAssets: 0.5934426229508196
CurrentCapitalLeaseObligation: 0.6196721311475409
CurrentDebt: 0.62295081967

In [57]:
required_cols = ['asOfDate','symbol','AccountsPayable','AccountsReceivable','NetIncome',
                                                      'StockholdersEquity','Inventory','GrossProfit',
                                                      'OperatingRevenue','TotalRevenue','OperatingCashFlow',
                                                      'TotalDebt','RetainedEarnings','DilutedEPS',
                                                      'DepreciationAndAmortizationInIncomeStatement',
                                                      'WorkingCapital','EBIT','TotalAssets','CommonStockEquity',
                                                      'TreasurySharesNumber','ShareIssued','MarketCap'
                                                      ]

In [58]:
req_fin_data = financial_data[required_cols]

In [59]:
req_fin_data

,asOfDate,symbol,AccountsPayable,AccountsReceivable,NetIncome,StockholdersEquity,Inventory,GrossProfit,OperatingRevenue,TotalRevenue,...,RetainedEarnings,DilutedEPS,DepreciationAndAmortizationInIncomeStatement,WorkingCapital,EBIT,TotalAssets,CommonStockEquity,TreasurySharesNumber,ShareIssued,MarketCap
0,2023-09-30,ADANIENT.NS,2.387815e+11,7.585700e+10,NaN,3.667165e+11,7.519760e+10,NaN,NaN,NaN,...,NaN,2.000,NaN,-9.059110e+10,NaN,1.481502e+12,3.667165e+11,NaN,1.140001e+09,NaN
1,2023-12-31,ADANIENT.NS,NaN,NaN,1.888450e+10,NaN,NaN,1.150826e+11,2.505023e+11,2.505023e+11,...,NaN,16.100,NaN,NaN,2.956120e+10,NaN,NaN,NaN,NaN,3.247863e+12
2,2024-03-31,ADANIENT.NS,2.555850e+11,9.792930e+10,4.493500e+09,3.907609e+11,9.486860e+10,5.324120e+10,2.002399e+11,2.002399e+11,...,1.862906e+11,NaN,NaN,-7.624790e+10,1.371280e+10,1.607318e+12,3.907609e+11,0.0,1.140001e+09,3.640651e+12
3,2024-06-30,ADANIENT.NS,NaN,NaN,1.454500e+10,NaN,NaN,1.020751e+11,2.547240e+11,2.547240e+11,...,NaN,12.300,NaN,NaN,3.366050e+10,NaN,NaN,NaN,NaN,3.620131e+12
4,2024-09-30,ADANIENT.NS,2.259388e+11,6.921470e+10,1.741750e+10,4.189571e+11,1.089123e+11,9.918050e+10,2.260807e+11,2.260807e+11,...,NaN,14.820,NaN,-1.596599e+11,3.318720e+10,1.817676e+12,4.189571e+11,NaN,1.140001e+09,3.573619e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2023-12-31,WIPRO.NS,NaN,NaN,2.694200e+10,NaN,NaN,6.822500e+10,2.220510e+11,2.220510e+11,...,NaN,2.580,NaN,NaN,3.864600e+10,NaN,NaN,NaN,NaN,2.460824e+12
2,2024-03-31,WIPRO.NS,2.327500e+10,1.154770e+11,2.834600e+10,7.498830e+11,9.070000e+08,6.486400e+10,2.220830e+11,2.220830e+11,...,6.309360e+11,NaN,NaN,3.982040e+11,4.193000e+10,1.152459e+12,7.498830e+11,11905480.0,1.045028e+10,2.505300e+12
3,2024-06-30,WIPRO.NS,2.220300e+10,1.140710e+11,3.003200e+10,7.800930e+11,9.120000e+08,6.633200e+10,2.196380e+11,2.196380e+11,...,6.614950e+11,2.865,NaN,4.371540e+11,4.350400e+10,1.190816e+12,7.800930e+11,11905480.0,1.046033e+10,2.689922e+12
4,2024-09-30,WIPRO.NS,1.976500e+10,1.126550e+11,3.208800e+10,8.183430e+11,1.052000e+09,6.796700e+10,2.230160e+11,2.230160e+11,...,6.936880e+11,3.060,NaN,4.755380e+11,4.634700e+10,1.260097e+12,8.183430e+11,11905480.0,1.046277e+10,2.828787e+12


In [60]:
req_fin_data.isna().sum()/len(req_fin_data)

asOfDate                                        0.000000
symbol                                          0.000000
AccountsPayable                                 0.534426
AccountsReceivable                              0.567213
NetIncome                                       0.180328
StockholdersEquity                              0.445902
Inventory                                       0.616393
GrossProfit                                     0.377049
OperatingRevenue                                0.180328
TotalRevenue                                    0.180328
OperatingCashFlow                               0.950820
TotalDebt                                       0.465574
RetainedEarnings                                0.793443
DilutedEPS                                      0.183607
DepreciationAndAmortizationInIncomeStatement    0.475410
WorkingCapital                                  0.593443
EBIT                                            0.344262
TotalAssets                    

In [61]:
for col in req_fin_data.columns:
    print(f"\nTickers with null {col}:")
    print(req_fin_data[req_fin_data[col].isna()]['symbol'].value_counts())


Tickers with null asOfDate:
Series([], Name: count, dtype: int64)

Tickers with null symbol:
Series([], Name: count, dtype: int64)

Tickers with null AccountsPayable:
symbol
INFY.NS          6
HDFCLIFE.NS      6
SBIN.NS          5
KOTAKBANK.NS     5
ICICIBANK.NS     5
SBILIFE.NS       5
INDUSINDBK.NS    5
AXISBANK.NS      5
NESTLEIND.NS     4
TATASTEEL.NS     3
TATAMOTORS.NS    3
ULTRACEMCO.NS    3
LT.NS            3
TRENT.NS         3
M&M.NS           3
MARUTI.NS        3
NTPC.NS          3
ONGC.NS          3
POWERGRID.NS     3
RELIANCE.NS      3
TITAN.NS         3
JSWSTEEL.NS      3
TECHM.NS         3
SUNPHARMA.NS     3
TCS.NS           3
TATACONSUM.NS    3
SHRIRAMFIN.NS    3
ADANIENT.NS      3
ADANIPORTS.NS    3
ITC.NS           3
APOLLOHOSP.NS    3
ASIANPAINT.NS    3
BAJAJ-AUTO.NS    3
BAJFINANCE.NS    3
BAJAJFINSV.NS    3
BEL.NS           3
BPCL.NS          3
BRITANNIA.NS     3
CIPLA.NS         3
COALINDIA.NS     3
EICHERMOT.NS     3
GRASIM.NS        3
HCLTECH.NS       3
HDFCBANK

In [62]:
for col in req_fin_data.columns:
    print(f"\nTickers with null {col}:")
    print(req_fin_data[req_fin_data[col].isna()]['asOfDate'].value_counts())


Tickers with null asOfDate:
Series([], Name: count, dtype: int64)

Tickers with null symbol:
Series([], Name: count, dtype: int64)

Tickers with null AccountsPayable:
asOfDate
2023-12-31    48
2024-06-30    47
2024-12-31    47
2023-09-30    11
2024-09-30     8
2025-03-31     1
2024-03-31     1
Name: count, dtype: int64

Tickers with null AccountsReceivable:
asOfDate
2023-12-31    48
2024-06-30    47
2024-12-31    47
2023-09-30    13
2024-09-30    10
2024-03-31     7
2025-03-31     1
Name: count, dtype: int64

Tickers with null NetIncome:
asOfDate
2023-09-30    48
2023-12-31     2
2024-06-30     2
2024-03-31     1
2024-09-30     1
2024-12-31     1
Name: count, dtype: int64

Tickers with null StockholdersEquity:
asOfDate
2023-12-31    43
2024-06-30    41
2024-12-31    41
2023-09-30     9
2024-03-31     1
2024-09-30     1
Name: count, dtype: int64

Tickers with null Inventory:
asOfDate
2023-12-31    50
2024-06-30    49
2024-12-31    49
2023-09-30    14
2024-03-31    13
2024-09-30    12
2

In [63]:
# req_fin_data.to_csv('req_fin_data.csv')
# req_fin_data = pd.read_csv(r'C:\Users\Preetham S Gowda\Desktop\Algo Trading\Market Research\Factor Investing\factor model\req_fin_data.csv')

In [64]:
req_fin_data = req_fin_data.sort_values(['symbol', 'asOfDate'])
req_fin_data = req_fin_data.groupby('symbol', group_keys=False).apply(lambda x: x.ffill())


In [65]:
req_fin_data = req_fin_data[(req_fin_data['symbol']!='INFY.NS')]

In [66]:
req_fin_data.isna().sum()/len(req_fin_data)

asOfDate                                        0.000000
symbol                                          0.000000
AccountsPayable                                 0.064626
AccountsReceivable                              0.159864
NetIncome                                       0.163265
StockholdersEquity                              0.037415
Inventory                                       0.238095
GrossProfit                                     0.367347
OperatingRevenue                                0.163265
TotalRevenue                                    0.163265
OperatingCashFlow                               0.921769
TotalDebt                                       0.061224
RetainedEarnings                                0.329932
DilutedEPS                                      0.006803
DepreciationAndAmortizationInIncomeStatement    0.452381
WorkingCapital                                  0.241497
EBIT                                            0.333333
TotalAssets                    

In [148]:
req_fin_data['yearmonth'] = req_fin_data['asOfDate'].dt.to_period('M')

In [235]:
logging.basicConfig(filename='indicator_errors.log', level=logging.ERROR)

def compute_technical_indicators(symbols: list[str]) -> pd.DataFrame:
    """
    Compute technical indicators for a list of stock symbols using 5 years of daily data.
    
    Args:
        symbols: List of stock ticker symbols (e.g., ['AAPL', 'MSFT']).
    
    Returns:
        A pandas DataFrame with technical indicators for each symbol, indexed by date and symbol.
    
    Raises:
        ValueError: If no valid data is retrieved for any symbol.
    """
    all_data = []

    for symbol in symbols:
        try:
            ticker = Ticker(symbol)
            hist = ticker.history(period='5y', interval='1d')
            if hist.empty or 'close' not in hist.columns or len(hist) < 252:
                logging.warning(f"Skipping {symbol}: Insufficient or invalid data")
                continue

            # Reset index and rename 'date' to 'asOfDate'
            hist = hist.reset_index()
            hist = hist.rename(columns={'date': 'asOfDate'})

            # Ensure 'asOfDate' is timezone-naive
            if hist['asOfDate'].dtype == 'datetime64[ns, tz]':  # Check for timezone-aware
                hist['asOfDate'] = hist['asOfDate'].dt.tz_localize(None)
            else:
                hist['asOfDate'] = pd.to_datetime(hist['asOfDate'], utc=True).dt.tz_localize(None)

            hist['symbol'] = symbol
            # hist.fillna(inplace=True).ffill()
            hist.sort_values('asOfDate', inplace=True)

            # Indicators
            hist['close_60'] = hist['close'].shift(60)
            hist['close_20'] = hist['close'].shift(20)
            hist['volume_20_sum'] = hist['volume'].rolling(window=20,min_periods=1).sum()
            hist['MA36'] = hist['close'].rolling(window=36,min_periods=1).mean()
            hist['close_36'] = hist['close'].shift(36)
            hist['volatility_60'] = hist['close'].rolling(window=60,min_periods=1).std()
            hist['volatility_20'] = hist['close'].rolling(window=20,min_periods=1).std()
            hist['volume_60_avg'] = hist['volume'].rolling(window=60,min_periods=1).mean()
            hist['volume_minus_avg'] = hist['volume'] - hist['volume_60_avg']
            hist['ema36_vol'] = hist['volume'].ewm(span=36).mean()
            hist['ema72_vol'] = hist['volume'].ewm(span=72).mean()
            hist['ATR42'] = (hist['high'] - hist['low']).rolling(window=42,min_periods=1).mean()
            hist['ema36_close'] = hist['close'].ewm(span=36).mean()
            hist['ema72_close'] = hist['close'].ewm(span=72).mean()
            ema1 = hist['close'].ewm(span=30, adjust=False).mean()
            ema2 = ema1.ewm(span=30, adjust=False).mean()
            ema3 = ema2.ewm(span=30, adjust=False).mean()
            hist['trix30'] = ema3.pct_change() * 100

            # Bollinger Bands
            hist['bollinger_mid'] = hist['close'].rolling(window=20,min_periods=1).mean()
            hist['bollinger_std'] = hist['close'].rolling(window=20,min_periods=1).std()
            hist['bollinger_up'] = hist['bollinger_mid'] + 2 * hist['bollinger_std']
            hist['bollinger_down'] = hist['bollinger_mid'] - 2 * hist['bollinger_std']

            # MFI42
            typical_price = (hist['high'] + hist['low'] + hist['close']) / 3
            money_flow = typical_price * hist['volume']
            positive_flow = np.where(typical_price > typical_price.shift(1), money_flow, 0)
            negative_flow = np.where(typical_price < typical_price.shift(1), money_flow, 0)
            pos_mf = pd.Series(positive_flow).rolling(window=42,min_periods=1).sum()
            neg_mf = pd.Series(negative_flow).rolling(window=42,min_periods=1).sum()
            hist['MFI42'] = 100 - (100 / (1 + pos_mf / (neg_mf + 1e-9)))  # Avoid division by zero

            # Returns
            hist['returns'] = hist['close'].pct_change()
            hist['returns_market'] = hist['returns']  # TODO: Replace with actual market index data
            hist['close_252'] = hist['close'].shift(252)
            hist['returns_60day'] = (hist['close'] - hist['close_60']) / hist['close_60']

            all_data.append(hist[['asOfDate', 'symbol', 'close', 'close_60', 'close_20', 'volume_20_sum',
                                  'MA36', 'close_36', 'volatility_60', 'volatility_20', 'volume_60_avg',
                                  'volume_minus_avg', 'ema36_vol', 'ema72_vol', 'ATR42', 'ema36_close',
                                  'ema72_close', 'bollinger_up', 'bollinger_down', 'MFI42', 'returns',
                                  'returns_market', 'close_252', 'returns_60day','trix30','volume']])

        except Exception as e:
            logging.error(f"Error processing symbol {symbol}: {e}")
            continue

    if not all_data:
        raise ValueError("No valid data retrieved for any symbol")

    # Ensure consistent timezone handling during concatenation
    indicator_df = pd.concat(all_data, ignore_index=True)
    indicator_df['asOfDate'] = indicator_df['asOfDate'].dt.tz_localize(None)  # Final check

    return indicator_df

In [236]:
tech_data = compute_technical_indicators(symbols=symbols)

In [237]:
tech_data.to_csv('tech_data.csv')
# tech_data = pd.read_csv('tech_data.csv')

In [238]:
tech_data['asOfDate'] = pd.to_datetime(tech_data['asOfDate'])

In [239]:
tech_data['yearmonth'] = pd.to_datetime(tech_data['asOfDate']).dt.to_period('M')

In [240]:
# Create a year-month column for grouping
tech_data['yearmonth'] = tech_data['asOfDate'].dt.to_period('M')

# Sort by date to ensure "last" makes sense
tech_data = tech_data.sort_values(by=['symbol', 'yearmonth', 'asOfDate'])

# Get the last row for each symbol-yearmonth combo
req_tech_data = tech_data.groupby(['symbol', 'yearmonth'], as_index=False).tail(1)

In [241]:
merged_df = pd.merge(req_fin_data, req_tech_data, 
                     how='left', 
                     on=['yearmonth', 'symbol'])

In [242]:
merged_df

,asOfDate_x,symbol,AccountsPayable,AccountsReceivable,NetIncome,StockholdersEquity,Inventory,GrossProfit,OperatingRevenue,TotalRevenue,...,ema72_close,bollinger_up,bollinger_down,MFI42,returns,returns_market,close_252,returns_60day,trix30,volume
0,2023-09-30,ADANIENT.NS,2.387815e+11,7.585700e+10,NaN,3.667165e+11,7.519760e+10,NaN,NaN,NaN,...,2451.564795,2577.673339,2412.741676,60.754861,-0.003242,-0.003242,3716.300049,0.036667,0.113350,2225560
1,2023-12-31,ADANIENT.NS,2.387815e+11,7.585700e+10,1.888450e+10,3.667165e+11,7.519760e+10,1.150826e+11,2.505023e+11,2.505023e+11,...,2546.762997,3165.455911,2445.734103,70.954237,-0.011764,-0.011764,4165.299805,0.135198,0.249302,4102912
2,2024-03-31,ADANIENT.NS,2.555850e+11,9.792930e+10,4.493500e+09,3.907609e+11,9.486860e+10,5.324120e+10,2.002399e+11,2.002399e+11,...,3028.679888,3387.710836,2941.404130,67.501870,0.003907,0.003907,1843.800049,0.111107,0.228232,900231
3,2024-06-30,ADANIENT.NS,2.555850e+11,9.792930e+10,1.454500e+10,3.907609e+11,9.486860e+10,1.020751e+11,2.547240e+11,2.547240e+11,...,3140.240703,3492.143866,2966.921173,52.686668,0.001467,0.001467,2485.649902,0.016992,0.072271,5114980
4,2024-09-30,ADANIENT.NS,2.259388e+11,6.921470e+10,1.741750e+10,4.189571e+11,1.089123e+11,9.918050e+10,2.260807e+11,2.260807e+11,...,3068.956887,3132.587780,2896.722206,57.316573,0.002450,0.002450,2540.050049,-0.019007,-0.073162,2297441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,2023-12-31,WIPRO.NS,7.885700e+10,1.081460e+11,2.694200e+10,6.989110e+11,1.281000e+09,6.822500e+10,2.220510e+11,2.220510e+11,...,207.763104,238.089891,194.837611,72.752362,-0.003185,-0.003185,194.475006,0.158135,0.090387,16839956
290,2024-03-31,WIPRO.NS,2.327500e+10,1.154770e+11,2.834600e+10,7.498830e+11,9.070000e+08,6.486400e+10,2.220830e+11,2.220830e+11,...,242.336126,268.914898,238.700103,60.602148,-0.015734,-0.015734,186.425003,0.006071,0.259620,14320668
291,2024-06-30,WIPRO.NS,2.220300e+10,1.140710e+11,3.003200e+10,7.800930e+11,9.120000e+08,6.633200e+10,2.196380e+11,2.196380e+11,...,236.624353,260.123940,215.111059,64.801147,0.031502,0.031502,194.475006,0.081516,-0.035332,42137138
292,2024-09-30,WIPRO.NS,1.976500e+10,1.126550e+11,3.208800e+10,8.183430e+11,1.052000e+09,6.796700e+10,2.230160e+11,2.230160e+11,...,259.137754,278.283253,255.589244,56.390645,-0.000185,-0.000185,220.524994,0.005195,0.113461,26462384


In [243]:
# merged_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [244]:
merged_df.isna().sum()/len(merged_df)

asOfDate_x                                      0.000000
symbol                                          0.000000
AccountsPayable                                 0.064626
AccountsReceivable                              0.159864
NetIncome                                       0.163265
StockholdersEquity                              0.037415
Inventory                                       0.238095
GrossProfit                                     0.367347
OperatingRevenue                                0.163265
TotalRevenue                                    0.163265
OperatingCashFlow                               0.921769
TotalDebt                                       0.061224
RetainedEarnings                                0.329932
DilutedEPS                                      0.006803
DepreciationAndAmortizationInIncomeStatement    0.452381
WorkingCapital                                  0.241497
EBIT                                            0.333333
TotalAssets                    

# Null Value Treatment

In [245]:
nifty_50['symbol'] = nifty_50['Symbol'] + '.NS'

In [246]:
final_df = merged_df.merge(nifty_50,on='symbol',how = 'left')

In [247]:
final_df

,asOfDate_x,symbol,AccountsPayable,AccountsReceivable,NetIncome,StockholdersEquity,Inventory,GrossProfit,OperatingRevenue,TotalRevenue,...,returns_market,close_252,returns_60day,trix30,volume,Company Name,Industry,Symbol,Series,ISIN Code
0,2023-09-30,ADANIENT.NS,2.387815e+11,7.585700e+10,NaN,3.667165e+11,7.519760e+10,NaN,NaN,NaN,...,-0.003242,3716.300049,0.036667,0.113350,2225560,Adani Enterprises Ltd.,Metals & Mining,ADANIENT,EQ,INE423A01024
1,2023-12-31,ADANIENT.NS,2.387815e+11,7.585700e+10,1.888450e+10,3.667165e+11,7.519760e+10,1.150826e+11,2.505023e+11,2.505023e+11,...,-0.011764,4165.299805,0.135198,0.249302,4102912,Adani Enterprises Ltd.,Metals & Mining,ADANIENT,EQ,INE423A01024
2,2024-03-31,ADANIENT.NS,2.555850e+11,9.792930e+10,4.493500e+09,3.907609e+11,9.486860e+10,5.324120e+10,2.002399e+11,2.002399e+11,...,0.003907,1843.800049,0.111107,0.228232,900231,Adani Enterprises Ltd.,Metals & Mining,ADANIENT,EQ,INE423A01024
3,2024-06-30,ADANIENT.NS,2.555850e+11,9.792930e+10,1.454500e+10,3.907609e+11,9.486860e+10,1.020751e+11,2.547240e+11,2.547240e+11,...,0.001467,2485.649902,0.016992,0.072271,5114980,Adani Enterprises Ltd.,Metals & Mining,ADANIENT,EQ,INE423A01024
4,2024-09-30,ADANIENT.NS,2.259388e+11,6.921470e+10,1.741750e+10,4.189571e+11,1.089123e+11,9.918050e+10,2.260807e+11,2.260807e+11,...,0.002450,2540.050049,-0.019007,-0.073162,2297441,Adani Enterprises Ltd.,Metals & Mining,ADANIENT,EQ,INE423A01024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,2023-12-31,WIPRO.NS,7.885700e+10,1.081460e+11,2.694200e+10,6.989110e+11,1.281000e+09,6.822500e+10,2.220510e+11,2.220510e+11,...,-0.003185,194.475006,0.158135,0.090387,16839956,Wipro Ltd.,Information Technology,WIPRO,EQ,INE075A01022
290,2024-03-31,WIPRO.NS,2.327500e+10,1.154770e+11,2.834600e+10,7.498830e+11,9.070000e+08,6.486400e+10,2.220830e+11,2.220830e+11,...,-0.015734,186.425003,0.006071,0.259620,14320668,Wipro Ltd.,Information Technology,WIPRO,EQ,INE075A01022
291,2024-06-30,WIPRO.NS,2.220300e+10,1.140710e+11,3.003200e+10,7.800930e+11,9.120000e+08,6.633200e+10,2.196380e+11,2.196380e+11,...,0.031502,194.475006,0.081516,-0.035332,42137138,Wipro Ltd.,Information Technology,WIPRO,EQ,INE075A01022
292,2024-09-30,WIPRO.NS,1.976500e+10,1.126550e+11,3.208800e+10,8.183430e+11,1.052000e+09,6.796700e+10,2.230160e+11,2.230160e+11,...,-0.000185,220.524994,0.005195,0.113461,26462384,Wipro Ltd.,Information Technology,WIPRO,EQ,INE075A01022


In [248]:
# Identify columns with missing values
null_cols = final_df.columns[final_df.isna().any()]

# Fill missing values with industry-wise mean
for col in null_cols:
    if col == 'Industry':
        continue
    final_df[col] = final_df.groupby('Industry')[col].transform(lambda x: x.fillna(x.mean()))


In [249]:
final_df.fillna(final_df.mean(numeric_only=True), inplace=True)

In [250]:
final_df.isna().sum()/len(final_df)

asOfDate_x                                      0.0
symbol                                          0.0
AccountsPayable                                 0.0
AccountsReceivable                              0.0
NetIncome                                       0.0
StockholdersEquity                              0.0
Inventory                                       0.0
GrossProfit                                     0.0
OperatingRevenue                                0.0
TotalRevenue                                    0.0
OperatingCashFlow                               0.0
TotalDebt                                       0.0
RetainedEarnings                                0.0
DilutedEPS                                      0.0
DepreciationAndAmortizationInIncomeStatement    0.0
WorkingCapital                                  0.0
EBIT                                            0.0
TotalAssets                                     0.0
CommonStockEquity                               0.0
TreasuryShar

# Variable Creation

In [251]:
# Ensure data is sorted by date for accurate calculation
final_df.sort_values(by=['symbol', 'asOfDate_x'],inplace=True)

### Quality Factors

In [252]:
final_df['ROE']=final_df['NetIncome']/final_df['StockholdersEquity']
final_df['DECM']=(final_df['AccountsReceivable'].diff() - final_df['Inventory'].diff())/final_df['TotalAssets']
final_df['GMI'] = final_df['GrossProfit'].diff()
final_df['ACCA'] = (final_df['NetIncome'] - final_df['OperatingCashFlow'])/final_df['TotalAssets']


### Value Factors

In [253]:
final_df['Debt2Equity']=final_df['TotalDebt']/final_df['StockholdersEquity']
final_df['CFO2Price'] = final_df['OperatingCashFlow']/final_df['close']

### Momentum Factors

In [254]:
#  trix30,volume_20_sum and ROC60 already created
final_df['PLRC36'] = (final_df['MA36']/final_df['close_36']) - 1
final_df['Price3M'] = final_df['close'] - final_df['close_60']


### Growth Factors

In [255]:
# PEG Ratio
final_df['PE'] = final_df['close'] / final_df['DilutedEPS']
final_df['earnings_growth'] = final_df.groupby('symbol')['NetIncome'].pct_change()
final_df['PEG'] = final_df['PE'] / final_df['earnings_growth']
final_df['PEG'] = final_df.groupby('symbol')['PEG'].ffill()
industry_avg_peg = final_df.groupby('Industry')['PEG'].transform('mean')
final_df['PEG'] = final_df['PEG'].fillna(industry_avg_peg)

# NetProfit Growth
final_df['net_profit_growth'] = final_df.groupby('symbol')['NetIncome'].pct_change()
industry_avg_net_profit_growth = final_df.groupby('Industry')['net_profit_growth'].transform('mean')
final_df['net_profit_growth'] = final_df['net_profit_growth'].fillna(industry_avg_net_profit_growth)

# EPS growth rate
final_df['eps_growth'] = final_df.groupby('symbol')['DilutedEPS'].pct_change()
industry_avg_eps_growth = final_df.groupby('Industry')['eps_growth'].transform('mean')
final_df['eps_growth'] = final_df['eps_growth'].fillna(industry_avg_eps_growth)

# Operating Revenue growth
final_df['operating_revenue_growth'] = final_df.groupby('symbol')['OperatingRevenue'].pct_change()
industry_avg_operating_revenue_growth = final_df.groupby('Industry')['operating_revenue_growth'].transform('mean')
final_df['operating_revenue_growth'] = final_df['operating_revenue_growth'].fillna(industry_avg_operating_revenue_growth)

# growth rate of operating cash flow
final_df['operating_cf_growth'] = final_df.groupby('symbol')['OperatingCashFlow'].pct_change()
industry_avg_operating_cf_growth_growth = final_df.groupby('Industry')['operating_cf_growth'].transform('mean')
final_df['operating_cf_growth'] = final_df['operating_cf_growth'].fillna(industry_avg_operating_cf_growth_growth)



### Emotional factors

In [256]:
# volatility_60, volume_60_avg, volume_minus_avg, ATR42 are already created
final_df['VMACD'] = final_df['ema36_vol'] - final_df['ema72_vol']

### Technical Factors

In [257]:
# Bollinger up down, MFI42 are created already
final_df['VMACD'] = final_df['ema36_close'] - final_df['ema72_close']

### Style factors

In [258]:
final_df['Book2Price'] = final_df['CommonStockEquity']/final_df['close']
final_df['Liquidity'] = final_df['volume']/(final_df['ShareIssued']-final_df['TreasurySharesNumber'])
final_df['Growth'] = np.log(final_df['close']/final_df['close_252'])
final_df['momentum'] = final_df['close']/final_df['close_252']


### Risk Factors

In [ ]:
RiskFree = 6.427

final_df['Variance60'] = final_df['volatility_20']*final_df['volatility_20']
final_df['Sharpe60'] = (final_df['returns'].mean() - RiskFree)/final_df['volatility_20']

In [180]:
final_df.columns

Index(['asOfDate_x', 'symbol', 'AccountsPayable', 'AccountsReceivable',
       'NetIncome', 'StockholdersEquity', 'Inventory', 'GrossProfit',
       'OperatingRevenue', 'TotalRevenue', 'OperatingCashFlow', 'TotalDebt',
       'RetainedEarnings', 'DilutedEPS',
       'DepreciationAndAmortizationInIncomeStatement', 'WorkingCapital',
       'EBIT', 'TotalAssets', 'CommonStockEquity', 'TreasurySharesNumber',
       'ShareIssued', 'MarketCap', 'yearmonth', 'asOfDate_y', 'close',
       'close_60', 'close_20', 'volume_20_sum', 'MA36', 'close_36',
       'volatility_60', 'volatility_20', 'volume_60_avg', 'volume_minus_avg',
       'ema36_vol', 'ema72_vol', 'ATR42', 'ema36_close', 'ema72_close',
       'bollinger_up', 'bollinger_down', 'MFI42', 'returns', 'returns_market',
       'close_252', 'returns_60day', 'Company Name', 'Industry', 'Symbol',
       'Series', 'ISIN Code'],
      dtype='object')